In [1]:
import time
import pandas as pd
import altair as alt
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor

from xgboost import XGBRegressor

# import other functions
from imputer import *
from feature_eng import *
from drop import *

In [2]:
def add_lagged_target(df):
    """
    This function takes a dataframe with the columns 
    "external_id", "month", "year" as primary keys.
    Adds a column of the "unacast_seassion_count" at lag of 1, as "session_lagged_1",
    and then deletes the first occurrence for each playground.


    Parameters
    ----------------
    df : pd.DataFrame
       A dataframe containing the columns "external_id", "month", "year"
       and "unacast_seassion_count"


    Returns
    ----------------
    pd.DataFrame
        With the new lagged session column, and deleted first occurrence of each playground, sorted by ["external_id","year","month"]
    """
    # subset and sort
    lagged = df.loc[:,["external_id","month","year","unacast_session_count"]].sort_values(by=["external_id","year","month"])

    # creat new column shifted by one (after sorting)
    lagged["session_lagged_1"] = lagged['unacast_session_count'].shift(1)
    
    # join the new column into the general dataframe
    out = pd.merge(df,lagged,how='left', left_on=['external_id','month','year',"unacast_session_count"], right_on =['external_id','month','year',"unacast_session_count"]).sort_values(by=["external_id","year","month"])

    # identify the 1st row of each playground, and subset as a df
    to_del = out.sort_values(by=["external_id","year","month"]).groupby('external_id',as_index=False).nth(0)
    # join the df with the rows to delete, then delete all rows that are have duplicates (both copies)
    out = pd.concat([out,to_del]).drop_duplicates(keep=False)
    return out

In [3]:
def report_performance(model, X_train, y_train, X_valid, y_valid, 
                       mode='mean'):
    """
    Evaluate train and validation performance on a fitted model.
    
    Parameters
    ---------     
    model: sklearn.ensemble._gb.GradientBoostingRegressor
        scikit-learn model
    X_train: pandas.core.frame.DataFrame
        X of training set
    y_train: pandas.core.series.Series
        y of training set
    X_valid: pandas.core.frame.DataFrame        
        X of validation set
    y_valid: pandas.core.series.Series
        y of validation set     
    mode: string
        'mean' or 'median'
    
    Returns
    -------
    errors: list
        
    """
    if mode == 'mean':
        errors = [mean_squared_error(y_train, 
                                     model.predict(X_train)) ** 0.5, 
                  mean_squared_error(y_valid, 
                                     model.predict(X_valid)) ** 0.5]
        
        print('Training RMSE:', errors[0])
        print('Validation RMSE:', errors[1])
        
        
    elif mode == 'median':
        errors = [mean_absolute_error(y_train, 
                                      model.predict(X_train)), 
                  mean_absolute_error(y_valid, 
                                      model.predict(X_valid))]
        
        print('Training MAE:', errors[0])
        print('Validation MAE:', errors[1])

In [4]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [56]:
df = pd.read_csv('../data/train_data.zip')

In [57]:
df = df.sort_values(by=['year', 'month'])

In [59]:
# create lagged target variable column
df = add_lagged_target(df)

In [60]:
# impute missing lagged (t-1) target variable
df['session_lagged_1'] = df.groupby('external_id')['session_lagged_1'].apply(lambda x: x.interpolate(limit_direction='both'))

In [61]:
# drop rows missing target variable
df = drop_missing_unacast(df)

In [67]:
# split data into `X_train` and `X_valid`

#train = df[(df['year'] != 2019) | (df['month'] <= 6)]
#valid = df[(df['year'] == 2019) & (df['month'] > 6)]

train = df.query("year != 2019 | month <= 6")
valid = df.query("year == 2019 & month > 6")

In [68]:
# create X_train and y_train
X_train = train.drop('unacast_session_count', axis=1)
y_train = train.loc[:, 'unacast_session_count']

In [69]:
# create X_valid and y_valid
X_valid = valid.drop('unacast_session_count', axis=1)
y_valid = valid.loc[:, 'unacast_session_count']

In [70]:
# impute NaN values
result = impute_data(X_train, X_valid)
X_train = result[0]
X_valid = result[1]

In [71]:
# perform feature engineering
X_train = comb_cols(X_train)
X_valid = comb_cols(X_valid)

In [72]:
# perform dropping
X_train = drop_columns(X_train)
X_valid = drop_columns(X_valid)

In [73]:
# OHE categorical columns on climate, density_class, income_class
X_train_valid = clean_categorical(X_train, X_valid)
X_train = X_train_valid[0]
X_valid = X_train_valid[1]

In [74]:
# evaluate basic model performance
lr = LinearRegression()
lr.fit(X_train, y_train)
report_performance(lr, X_train, y_train, X_valid, y_valid)

Training RMSE: 78.74329241363074
Validation RMSE: 171.80114353341787


In [75]:
# create `month_number` column to create indices to use in cross validation
months_df = df.query("external_id == '01a78f56-5cc9-4309-8676-057933848570'").loc[:, ['month', 'year']]
months_df = months_df.reset_index().drop(columns='index').reset_index()
months_df = months_df.rename(columns={'index':'cross_val_ind'})
months_df = months_df[:16]

In [39]:
X_train = pd.merge(X_train, months_df, on=['month', 'year'], how='left')

In [49]:
X_train[(X_train['cross_val_ind'] == 0) | (X_train['cross_val_ind'] == 1)]

,month,year,monthly_number_of_sessions,monthly_unique_sessions,monthly_repeated_sessions,monthly_avg_length_of_session,monthly_avg_light_activity,monthly_avg_moderate_activity,monthly_avg_vigorous_activity,monthly_count_ramp,...,HI,LI,MI,HD,LD,MD,A,C,D,cross_val_ind
0,3,2018,13,2,11,392153.846154,91923.076923,60769.230769,0.000000,0,...,0,0,1,1,0,0,1,0,0,0
1,4,2018,0,0,0,0.000000,0.000000,0.000000,0.000000,0,...,0,0,1,1,0,0,1,0,0,1
16,3,2018,0,0,0,0.000000,0.000000,0.000000,0.000000,0,...,0,1,0,0,1,0,0,1,0,0
17,4,2018,0,0,0,0.000000,0.000000,0.000000,0.000000,0,...,0,1,0,0,1,0,0,1,0,1
31,3,2018,0,0,0,0.000000,0.000000,0.000000,0.000000,0,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39587,4,2018,0,0,0,0.000000,0.000000,0.000000,0.000000,0,...,0,1,0,1,0,0,0,1,0,1
39602,3,2018,0,0,0,0.000000,0.000000,0.000000,0.000000,0,...,0,0,1,0,1,0,0,1,0,0
39603,4,2018,0,0,0,0.000000,0.000000,0.000000,0.000000,0,...,0,0,1,0,1,0,0,1,0,1
39618,3,2018,0,0,0,0.000000,0.000000,0.000000,0.000000,0,...,0,0,1,0,1,0,0,1,0,0


In [ ]:
# get indices for time dependent 

# split 1
# train: 2018-03 to 2018-04; test: 2018-05 to 2018-06

In [ ]:
# split 2
# train: 2018-03 to 2018-06; test: 2018-07 to 2018-09

# split 3
# train: 2018-03 to 2018-09; test: 2018-10 to 2018-12

# split 4
# train: 2018-03 to 2018-12; test: 2019-01 to 2019-03

# split 5
# train: 2018-03 to 2019-03; test: 2019-04 to 2019-06